In [408]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import copy, math
# scaler = StandardScaler()
import tensorflow as tf
from tensorflow import keras
# from recsys_utils import *

In [409]:
column_names = ['User ID', 'Movie ID', 'Rating', 'Timestamp']
df = pd.read_csv('../../notes and data/ml-100k/u.data',sep='\t', names=column_names)
df = df.drop(columns=['Timestamp'])
df

,User ID,Movie ID,Rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [410]:
y = df.pivot(index='Movie ID', columns='User ID', values='Rating')
y = y.fillna(0)
y = np.array(y)
y

array([[5., 4., 0., ..., 5., 0., 0.],
       [3., 0., 0., ..., 0., 0., 5.],
       [4., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [411]:
M = (y != 0).astype(int)
print(y.shape,M.shape)

(1682, 943) (1682, 943)


In [412]:
# def costFn(x,y,w,b,lam):
#     cost = 0
#     pred = 0
#     reg = 0
#     for i in range(y.shape[0]):
#         for j in range(y.shape[1]):
#             if m[i][j] == 1:
#                 val = np.dot(w[j],x[i]) + b[j] - y[i][j]
#                 pred += (val**2)/2
                
#     for i in range(y.shape[0]):  # Regularization for X
#         reg += np.sum(x[i] ** 2)

#     for j in range(y.shape[1]):  # Regularization for W and b
#         reg += np.sum(w[j] ** 2) + b[j] ** 2

#     reg = (lam/2)*reg

#     cost = pred + reg  # Divide by 2m for proper scaling
#     return cost


def costFn(X, Y,W, b,m, lam):
    
    nm, nu = Y.shape
    J = 0
    ### START CODE HERE ###  
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = m[i,j]
            J += r * np.square((np.dot(w,x) + b_j - y ))
    J += (lam) * (np.sum(np.square(W)) + np.sum(np.square(X)))
    J = J/2
    ### END CODE HERE ### 

    return J
    # j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*m
    # J = 0.5 * tf.reduce_sum(j**2) + (lam/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    # return J


In [413]:
# def gradientfn(x,y,w,b,m,lam):
#     djx = np.zeros_like(x)
#     djw = np.zeros_like(w)
#     djb = np.zeros_like(b)

#     for i in range(y.shape[0]):
#         for k in range(len(x[i])):
#             for j in range(y.shape[1]):
#                 if m[i][j] == 1:
#                     val = np.dot(w[j],x[i]) + b[0,j] - y[i][j]
#                     djx[i][k] += val*w[j][k]
#             djx[i][k] += lam*x[i][k]
            
#     for j in range(y.shape[1]):
#         for k in range(len(w[j])):
#             for i in range(y.shape[0]):
#                 if m[i][j] == 1:
#                     val = np.dot(w[j],x[i]) + b[0,j] - y[i][j]
#                     djw[j][k] += val*x[i][k]
#             djw[j][k] += lam*w[j][k]
            
#     for j in range(y.shape[1]):
#         for i in range(y.shape[0]):
#             if m[i][j] == 1:
#                 val = np.dot(w[j],x[i]) + b[0,j] - y[i][j]
#                 djb[0,j] += val    
    
#     return djw, djb ,djx 

def gradientfn(x, y, w, b, m, lam):
    m_samples = x.shape[0]  # Number of samples (users)
    n_items = y.shape[1]  # Number of items (movies)
    
    # Compute the error term (predictions - true values)
    error = (np.dot(x, w.T) + b - y) * m  # Apply mask to the error term
    
    # Gradients for X (scaled by the number of rated items and regularization)
    djx = (2 / m_samples) * np.dot(error, w) + lam * x  # Shape: (1682, 100)
    
    # Gradients for W (scaled by the number of rated items and regularization)
    djw = (2 / m_samples) * np.dot(error.T, x) + lam * w  # Shape: (40, 100)
    
    # Gradients for B (scaled by the number of rated items)
    djb = (2 / m_samples) * np.sum(error, axis=0)  # Shape: (1, 40)
    
    return djw, djb, djx



In [414]:
def gradientDecent(x,y,w_in,b_in,alpha,iters,lam,m):
    x = copy.deepcopy(x)
    w = copy.deepcopy(w_in) #avoid modifying global w within function
    b = copy.deepcopy(b_in)
    print("enter")
    for i in range(iters):
        djw,djb,djx = gradientfn(x,y,w,b,m,lam)
        w = w - alpha*djw
        b = b - alpha*djb
        x = x - alpha*djx
        if i % 100 == 0 and i > 0:
            alpha *= 0.9
        if i % math.ceil(iters / 10) == 0:
            cost = costFn(x,y,w,b,m,lam)
            # alpha *= 1.1
            print(f"Iteration {i:4d}: Cost {cost:.4f}")
    return w, b, x

In [415]:
def predict(xt, wt, bt):
    print(xt.shape,wt.shape,bt.shape)
    # predicted_ratings = np.dot(x, w.T) + b  # Matrix multiplication for all users and all movies
    predicted_ratings = np.matmul(xt, np.transpose(wt)) + bt
    return predicted_ratings

In [416]:
num_features = 100
num_movies, num_users = y.shape  # Total number of users (943)
subset_users = 500 # Number of users you want to use for faster processing
optimizer = keras.optimizers.Adam(learning_rate=0.5)
# Randomly select a subset of users
user_indices = np.arange(num_users)
selected_users = np.random.choice(user_indices, size=subset_users, replace=False)

# Split the selected users into training and test sets (80% train, 20% test)
train_users, test_users = train_test_split(selected_users, test_size=0.2, random_state=42)

# Select the corresponding data for the training and test sets
y_train = y[:, train_users]  
y_test = y[:, test_users]    

M_train = M[:, train_users]  
M_test = M[:, test_users]    

nm,nu = y_train.shape
print(nm,nu)

alpha = 0.7  # Reduce the learning rate to avoid divergence
iterations = 300
lam = 0.00001

# Initialize features and weights with small random values instead of zeros
# xi = np.random.randn(y_train.shape[0], 10) * 0.1  # Small random values
# wi = np.random.randn(y_train.shape[1], 10) * 0.1 # Small random values
# bi = np.random.randn(1,y_train.shape[1]) * 0.1  # Small random values
# print(initial_x.shape,initial_w.shape,initial_b.shape)

tf.random.set_seed(1234) # for consistent results

wi = tf.Variable(tf.random.normal((nu,  num_features),dtype=tf.float64),  name='w')
xi = tf.Variable(tf.random.normal((nm, num_features),dtype=tf.float64),  name='x')
bi = tf.Variable(tf.random.normal((1,          nu),   dtype=tf.float64),  name='b')
wi = wi.numpy()
bi = bi.numpy()
xi = xi.numpy()
print(w.shape,b.shape,x.shape)

1682 400
(400, 100) (1, 400) (1682, 100)


In [417]:
w,b,x = gradientDecent(xi,y_train,wi,bi,alpha,iterations,lam,M_train)

enter
Iteration    0: Cost 1078243.9828
Iteration   30: Cost 18351.4878
Iteration   60: Cost 9243.1028
Iteration   90: Cost 7046.6654
Iteration  120: Cost 6085.4722
Iteration  150: Cost 5486.4526
Iteration  180: Cost 5035.7947
Iteration  210: Cost 4680.0360
Iteration  240: Cost 4395.1019
Iteration  270: Cost 4142.5848


In [418]:
y_pred = predict(x, w, b)
y_pred_masked = y_pred * M_train
y_masked = y_train * M_train
y_flat = y_masked[M_train > 0]
y_pred_flat = y_pred_masked[M_train > 0]
mse = mean_squared_error(y_flat, y_pred_flat)
rmse = np.sqrt(mse)
print(y_flat)
print(y_pred_flat)
print(r2_score(y_flat, y_pred_flat))
print(mse,rmse)
print(r2_score(y_masked, y_pred_masked))

(1682, 100) (400, 100) (1, 400)
[2. 5. 4. ... 3. 2. 3.]
[2.24524159 4.75761987 4.14519347 ... 2.99031402 2.00065221 2.8928208 ]
0.863077978951342
0.17398074775918582 0.4171099947965594
0.9940062638126966


In [420]:
(1682, 100) (80, 100) (1, 80)
[4. 5. 5. ... 1. 5. 2.]
[3.3234566  5.02058092 5.25767756 ... 0.99529744 4.61865921 2.00393154]
0.9434746259993244
0.07191042153243683 0.26816118573059156
0.9974765094971586
lam 0.00001
alpha 0.7
with decay

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2233114766.py, line 2)

In [422]:
# Fix learned movie features from training
X_fixed = x  

# Initialize test users' parameters
w_test = tf.Variable(tf.random.normal((len(test_users), num_features), dtype=tf.float64), name='w_test')
b_test = tf.Variable(tf.random.normal((1, len(test_users)), dtype=tf.float64), name='b_test')
w_test = w_test.numpy()
b_test = b_test.numpy()

def gradientfn1(x, y, w, b, m, lam):
    m_samples = x.shape[0]  # Number of samples (users)
    n_items = y.shape[1]  # Number of items (movies)
    
    # Compute the error term (predictions - true values)
    error = (np.dot(x, w.T) + b - y) * m  # Apply mask to the error term
    
    # Gradients for W (scaled by the number of rated items and regularization)
    djw = (2 / m_samples) * np.dot(error.T, x) + lam * w  # Shape: (40, 100)
    
    # Gradients for B (scaled by the number of rated items)
    djb = (2 / m_samples) * np.sum(error, axis=0)  # Shape: (1, 40)
    
    return djw, djb

def gradientDecent1(x,y,w_in,b_in,alpha,iters,lam,m):
    x = copy.deepcopy(x)
    w = copy.deepcopy(w_in) #avoid modifying global w within function
    b = copy.deepcopy(b_in)
    print("enter")
    for i in range(iters):
        djw,djb,djx = gradientfn(x,y,w,b,m,lam)
        w = w - alpha*djw
        b = b - alpha*djb
        if i % 100 == 0 and i > 0:
            alpha *= 0.9
        if i % math.ceil(iters / 10) == 0:
            cost = costFn(x,y,w,b,m,lam)
            # alpha *= 1.1
            print(f"Iteration {i:4d}: Cost {cost:.4f}")
    return w, b

w_test,b_test = gradientDecent1(X_fixed,y_test,w_test,b_test,alpha,iterations,lam,M_test)
y_test_pred = np.matmul(X_fixed, np.transpose(w_test)) + b_test

# Apply mask and evaluate
y_test_pred_masked = y_test_pred * M_test
y_test_flat = y_test[M_test > 0]
y_test_pred_flat = y_test_pred_masked[M_test > 0]

mse_test = mean_squared_error(y_test_flat, y_test_pred_flat)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test_flat, y_test_pred_flat)
print(r2_score(y_test, y_test_pred_masked))
print(f"Test RMSE: {rmse_test:.4f}")
print(f"Test R^2 Score: {r2_test:.4f}")


enter
Iteration    0: Cost 282501.1131
Iteration   30: Cost 10927.1100
Iteration   60: Cost 4636.4846
Iteration   90: Cost 3034.0544
Iteration  120: Cost 2396.0643
Iteration  150: Cost 2059.3122
Iteration  180: Cost 1850.7414
Iteration  210: Cost 1715.6430
Iteration  240: Cost 1625.4596
Iteration  270: Cost 1557.6956
0.9846641752360071
Test RMSE: 0.5632
Test R^2 Score: 0.7388


In [425]:
y_test_pred.shape

(1682, 100)

In [428]:
for i in range(len(y_test_pred)):
    print(y_test_pred[i][0])

4.5669926314661975
5.014108689105486
-4.74603589573689
10.116195760001158
-0.7494487634824072
-8.393390376412095
1.8826897393145368
1.52070413115743
3.6838589325127633
-0.9461314201520636
2.5691288501945504
-0.25510027130560875
3.659309954091266
5.7619369233080935
3.8431838065911044
15.049209852410247
8.080362774217466
-0.7470148483890107
-2.389197867364108
8.870956353068665
1.455466476331932
4.491352671349185
10.10276526526776
4.038851625787972
4.848286348985901
18.950796228504036
4.65330822341956
-0.8103103998383596
11.546393934567885
13.275083357899659
1.5505227636290886
2.2525093653009294
-0.09898607088257494
6.46910736316297
6.361395994740719
-7.5248919968261525
-1.7290775069275486
5.548105537712664
-6.457917031985465
2.7541481682402496
0.5939485428529812
6.020091908136092
-3.567161214593823
10.551105634267469
9.194117661693998
-8.979822950516466
10.970548975049176
-10.003305411879866
3.0684259064553063
2.462371969474999
9.376701876459352
-1.8996587753536214
3.3331643284774253
-7.

In [430]:
# Assuming y_pred is your predicted ratings (shape: [num_movies, num_users])
# Assuming M_train is the mask indicating whether the user has rated a movie (1 = rated, 0 = not rated)

# For a specific user, say user 0:
user_index = 0
user_ratings = y_test_pred[:, user_index]  # Predicted ratings for all movies for user 0 (shape: [num_movies])
user_mask = M_test[:, user_index]    # Mask indicating which movies user 0 has rated (shape: [num_movies])

# Step 1: Multiply predicted ratings by the mask (set rated movies' predictions to 0)
user_ratings_masked = user_ratings * (1 - user_mask)  # This will set rated movies' predictions to 0

# Step 2: Create a dictionary with movie indices as keys and ratings as values
movie_rating_dict = {index: rating for index, rating in enumerate(user_ratings_masked)}

# Step 3: Sort the dictionary by ratings in descending order
sorted_movie_dict = dict(sorted(movie_rating_dict.items(), key=lambda item: item[1], reverse=True))

# Step 4: Filter out the movies that the user has already rated (you've already done this in Step 1)
# So the sorted_movie_dict already contains only the unrated movies, ordered by predicted rating

# Example: Print the top 10 recommended movies for user 0
top_n_recommendations = list(sorted_movie_dict.items())[:10]  # Top 10 recommended movies

print("Top 10 recommendations for User 0:")
for movie_index, predicted_rating in top_n_recommendations:
    print(f"Movie Index: {movie_index}, Predicted Rating: {predicted_rating}")

Top 10 recommendations for User 0:
Movie Index: 1112, Predicted Rating: 23.110177358206432
Movie Index: 1431, Predicted Rating: 21.738737662763338
Movie Index: 455, Predicted Rating: 20.311347475794832
Movie Index: 1107, Predicted Rating: 20.276893680970737
Movie Index: 737, Predicted Rating: 20.2378854940794
Movie Index: 1018, Predicted Rating: 19.84982937410345
Movie Index: 463, Predicted Rating: 19.806085743078928
Movie Index: 1284, Predicted Rating: 19.748694259873147
Movie Index: 660, Predicted Rating: 19.655042474709784
Movie Index: 1339, Predicted Rating: 19.61054736898552
